In [1]:
import os
import zipfile
import json
import pandas as pd

## trianing data 전처리
- 전체 데이터의 5퍼센트를 정답 데이터로 바꾸기
- original text와 summary text를 'text'행으로 묶어 tagging

디렉토리 정보
- input : '../data/original/training/'
- output : '../data/training/'

csv파일 명칭 정보
- training_machine_fact.csv : TL_기계요약문_사실전달형_corrected_type1 ~ TL_기계요약문_사실전달형_corrected_type6
- training_machine_opinion.csv : TL_기계요약문_의견제시형_corrected_type1 ~ TL_기계요약문_의견제시형_corrected_type6
- training_machine_free.csv : TL_기계요약문_자유형_corrected_type1 ~ TL_기계요약문_자유형_corrected_type6
- training_human_fact.csv : TL_사람요약문_사실전달형_corrected_type1 ~ TL_사람요약문_사실전달형_corrected_type6
- training_human_opinion.csv : TL_사람요약문_의견제시형_corrected_type1 ~ TL_사람요약문_의견제시형_corrected_type6
- training_human_free.csv : TL_사람요약문_자유형_corrected_type1 ~ TL_사람요약문_자유형_corrected_type6

In [1]:
# 압축을 풀 디렉토리 경로 설정
directory = '../data/original/training'

# 해당 디렉토리 내의 모든 파일을 순회
for filename in os.listdir(directory):
    # 파일이 zip 확장자를 가지고 있는 경우
    if filename.endswith('.zip'):
        # zip 파일의 전체 경로
        zip_path = os.path.join(directory, filename)
        # 압축 해제할 위치 (같은 디렉토리 내)
        extract_path = os.path.join(directory, filename[:-4])

        # zip 파일 열기
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # zip 파일 압축 해제
            zip_ref.extractall(extract_path)

        print(f'{filename} 압축 해제 완료.')

print('모든 zip 파일 압축 해제 완료.')


TL_사람요약문_자유형_corrected_type1.zip 압축 해제 완료.
TL_기계요약문_사실전달형_corrected_type4.zip 압축 해제 완료.
TL_사람요약문_의견제시형_corrected_type2.zip 압축 해제 완료.
TL_기계요약문_자유형_corrected_type6.zip 압축 해제 완료.
TL_사람요약문_의견제시형_corrected_type3.zip 압축 해제 완료.
TL_기계요약문_사실전달형_corrected_type5.zip 압축 해제 완료.
TL_사람요약문_자유형_corrected_type2.zip 압축 해제 완료.
TL_사람요약문_의견제시형_corrected_type1.zip 압축 해제 완료.
TL_기계요약문_자유형_corrected_type5.zip 압축 해제 완료.
TL_기계요약문_자유형_corrected_type4.zip 압축 해제 완료.
TL_기계요약문_사실전달형_corrected_type6.zip 압축 해제 완료.
TL_사람요약문_자유형_corrected_type3.zip 압축 해제 완료.
TL_기계요약문_사실전달형_corrected_type2.zip 압축 해제 완료.
TL_사람요약문_의견제시형_corrected_type4.zip 압축 해제 완료.
TL_기계요약문_자유형_corrected_type1.zip 압축 해제 완료.
TL_사람요약문_의견제시형_corrected_type5.zip 압축 해제 완료.
TL_기계요약문_사실전달형_corrected_type3.zip 압축 해제 완료.
TL_사람요약문_자유형_corrected_type6.zip 압축 해제 완료.
TL_사람요약문_자유형_corrected_type4.zip 압축 해제 완료.
TL_기계요약문_사실전달형_corrected_type1.zip 압축 해제 완료.
TL_기계요약문_자유형_corrected_type3.zip 압축 해제 완료.
TL_기계요약문_자유형_corrected_type2.zip 압축 해제 완료.
TL_사람요약문_의견제시형_corrected_type6.z

In [17]:
base_dir_path = '../data/original/training/'
base_output_dir_path = '../data/training/'
percent = 5

In [18]:
# TL_기계요약문_사실전달형_corrected_type1 ~ TL_기계요약문_사실전달형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    machine_type_data_path = base_dir_path + f'TL_기계요약문_사실전달형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(machine_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(machine_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['machine_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_machine_fact = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_machine_fact.shape)
print(data_df_training_machine_fact.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_machine_fact.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_training_machine_fact.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (28291, 2)
   label                                               text
0      0  [original]음성군이 여름방학을 맞은 어린이들에게 다양한 볼거리와 체험 행사를...
1      0  [original]창단 4년 만에 괄목할만한 대회 성적을 올리며 화제가 되고 있는 ...
2      0  [original]주방가전 전문 브랜드 하우스쿡이 가정의 달 이벤트를 진행한다. 5...
3      0  [original]대한축구협회(KFA)가 FIFA U-20 월드컵 준우승을 차지한 ...
4      0  [original]올해 2분기 전세계 스마트폰 판매량이 전년 대비 소폭 감소했다. ...
데이터가 저장되었습니다.


In [19]:
#training_machine_opinion.csv : TL_기계요약문_의견제시형_corrected_type1 ~ TL_기계요약문_의견제시형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    machine_type_data_path = base_dir_path + f'TL_기계요약문_의견제시형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(machine_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(machine_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['machine_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_machine_opinion = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_machine_opinion.shape)
print(data_df_training_machine_opinion.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_machine_opinion.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_training_machine_opinion.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (27735, 2)
   label                                               text
0      0  [original]한국GM 노사의 임단협 협상 데드라인이 당초 지난 20 일에서 2...
1      0  [original]정부가 논란의 중심에 서 있는 가상화폐에 대해 블록체인과 분리해 ...
2      0  [original]노동계가 5 월 춘투(春鬪)에 본격 나설 움직임이다. 올해 어느 ...
3      0  [original]경제 상황의 진단을 놓고 최고위 경제정책 당국자들 간에 불협화음이...
4      0  [original]정부가 백지화하기로 한 신한울 원전 3, 4 호기 건설을 재개해야...
데이터가 저장되었습니다.


In [20]:
#training_machine_free.csv : TL_기계요약문_자유형_corrected_type1 ~ TL_기계요약문_자유형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    machine_type_data_path = base_dir_path + f'TL_기계요약문_자유형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(machine_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(machine_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['machine_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_machine_free = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_machine_free.shape)
print(data_df_training_machine_free.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_machine_free.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_training_machine_free.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (16191, 2)
   label                                               text
0      0  [original]가. 기명주식의 양도는 양도약정당시 시행되던 상법(1962.1.2...
1      0  [original]1 지방자치단체가 설치경영하는 학교의 교장이나 교사는 학생을 보호...
2      0  [original]가. 사회안전법에 의한 법무부장관의 결정에 대한 이의소송에 관하여...
3      0  [original]1 상표법 제8조 제2항은 동일 또는 유사한 상품에 사용할 동일 ...
4      0  [original]가. 수익적 행정처분을 취소하거나 중지시키는 경우에는 이미 부여된...
데이터가 저장되었습니다.


In [21]:
#training_human_fact.csv : TL_사람요약문_사실전달형_corrected_type1 ~ TL_사람요약문_사실전달형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'TL_사람요약문_사실전달형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_human_fact = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_human_fact.shape)
print(data_df_training_human_fact.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_human_fact.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_training_human_fact.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (3627, 2)
   label                                               text
0      0  [original]제이테크놀로지(옛 마제스타)가 카지노 자회사 매각을 완료해 재무건...
1      0  [original]글로벌 뷰티 브랜드 랑콤이 패션 매거진 <마리끌레르>와 함께한 배...
2      0  [original]법동초 시작관내 초 순회 진단. 대덕구가 지난 30일 우송대학교 ...
3      0  [original]경기파주시 최종환시장은 지난 4일 신설 마을버스 운행 첫날 시민들...
4      0  [original]배터리 셀을 든 최태원(가운데) 회장이 김진영 배터리생산기술본부장...
데이터가 저장되었습니다.


In [22]:
#training_human_opinion.csv : TL_사람요약문_의견제시형_corrected_type1 ~ TL_사람요약문_의견제시형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'TL_사람요약문_의견제시형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_human_opinion = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_human_opinion.shape)
print(data_df_training_human_opinion.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_human_opinion.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_training_human_opinion.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (3412, 2)
   label                                               text
0      0  [original]고용노동부가 최근 국무회의에서 발표한 2011~2020 중장기 인...
1      0  [original]지난 3월 20일 서울시의회에서는 촛불혁명으로 어렵사리 진전시켜놓...
2      0  [original]일본의 경제보복 사태가 여전히 살얼음판이다. 일본이 금수 초지한 ...
3      0  [original]부산시가 해수담수화 시설 산업용수의 수요처로 예상했던 울산 온산미...
4      0  [original]북한 비핵화를 위한 하노이 선언이 무산됐다. 전 세계의 뜨거운 관...
데이터가 저장되었습니다.


In [23]:
#training_human_free.csv : TL_사람요약문_자유형_corrected_type1 ~ TL_사람요약문_자유형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'TL_사람요약문_자유형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_human_free = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_human_free.shape)
print(data_df_training_human_free.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_human_free.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_training_human_free.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (1871, 2)
   label                                               text
0      0  [original]가. 불법행위의 피해자가 입은 소극적 손해를 산정함에 있어 노동능...
1      0  [original]1 지방세법 제109조 제3항 본문은 토지구획정리사업법에 의한 토...
2      0  [original]가. 상표법 제43조 제1항 제1호에 의한 상표등록취소심판청구는 ...
3      0  [original]가. 제2차 납세의무자에 해당하는 과점주주인 여부는 과세처분을 한...
4      0  [original]가. 면제된 취득세, 등록세의 추징에 관한 규정인 구 지방세법(1...
데이터가 저장되었습니다.


## validation data 전처리
- 전체 데이터의 5퍼센트를 정답 데이터로 바꾸기
- original text와 summary text를 'text'행으로 묶어 tagging

디렉토리 정보
- input : '../data/original/validation/'
- output : '../data/validation/'

csv파일 명칭 정보
- validation_machine_fact.csv : VL_기계요약문_사실전달형_corrected_type1 ~ VL_기계요약문_사실전달형_corrected_type6
- validation_machine_opinion.csv : VL_기계요약문_의견제시형_corrected_type1 ~ VL_기계요약문_의견제시형_corrected_type6
- validation_machine_free.csv : VL_기계요약문_자유형_corrected_type1 ~ VL_기계요약문_자유형_corrected_type6
- validation_human_fact.csv : VL_사람요약문_사실전달형_corrected_type1 ~ VL_사람요약문_사실전달형_corrected_type6
- validation_human_opinion.csv : VL_사람요약문_의견제시형_corrected_type1 ~ VL_사람요약문_의견제시형_corrected_type6
- validation_human_free.csv : VL_사람요약문_자유형_corrected_type1 ~ VL_사람요약문_자유형_corrected_type6

In [25]:
# 압축을 풀 디렉토리 경로 설정
directory = '../data/original/validation'

# 해당 디렉토리 내의 모든 파일을 순회
for filename in os.listdir(directory):
    # 파일이 zip 확장자를 가지고 있는 경우
    if filename.endswith('.zip'):
        # zip 파일의 전체 경로
        zip_path = os.path.join(directory, filename)
        # 압축 해제할 위치 (같은 디렉토리 내)
        extract_path = os.path.join(directory, filename[:-4])

        # zip 파일 열기
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # zip 파일 압축 해제
            zip_ref.extractall(extract_path)

        print(f'{filename} 압축 해제 완료.')

print('모든 zip 파일 압축 해제 완료.')


VL_사람요약문_자유형_corrected_type4.zip 압축 해제 완료.
VL_기계요약문_사실전달형_corrected_type6.zip 압축 해제 완료.
VL_기계요약문_사실전달형_corrected_type4.zip 압축 해제 완료.
VL_기계요약문_사실전달형_corrected_type5.zip 압축 해제 완료.
VL_사람요약문_자유형_corrected_type6.zip 압축 해제 완료.
VL_사람요약문_자유형_corrected_type2.zip 압축 해제 완료.
VL_기계요약문_사실전달형_corrected_type1.zip 압축 해제 완료.
VL_사람요약문_자유형_corrected_type3.zip 압축 해제 완료.
VL_사람요약문_자유형_corrected_type1.zip 압축 해제 완료.
VL_기계요약문_사실전달형_corrected_type2.zip 압축 해제 완료.
VL_기계요약문_사실전달형_corrected_type3.zip 압축 해제 완료.
VL_사람요약문_사실전달형_corrected_type3.zip 압축 해제 완료.
VL_사람요약문_의견제시형_corrected_type1.zip 압축 해제 완료.
VL_사람요약문_사실전달형_corrected_type2.zip 압축 해제 완료.
VL_사람요약문_의견제시형_corrected_type3.zip 압축 해제 완료.
VL_사람요약문_의견제시형_corrected_type2.zip 압축 해제 완료.
VL_사람요약문_사실전달형_corrected_type1.zip 압축 해제 완료.
VL_사람요약문_사실전달형_corrected_type5.zip 압축 해제 완료.
VL_사람요약문_의견제시형_corrected_type6.zip 압축 해제 완료.
VL_사람요약문_사실전달형_corrected_type4.zip 압축 해제 완료.
VL_사람요약문_사실전달형_corrected_type6.zip 압축 해제 완료.
VL_사람요약문_의견제시형_corrected_type5.zip 압축 해제 완료.
VL_사람요약문_의견제시형_corre

In [3]:
base_dir_path = '../data/original/validation/'
base_output_dir_path = '../data/validation/'
percent = 5

In [12]:
# validation_machine_fact.csv : VL_기계요약문_사실전달형_corrected_type1 ~ VL_기계요약문_사실전달형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    machine_type_data_path = base_dir_path + f'VL_기계요약문_사실전달형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(machine_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(machine_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['machine_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_machine_fact = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_machine_fact.shape)
print(data_df_validation_machine_fact.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_machine_fact.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_validation_machine_fact.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (3537, 2)
   label                                               text
0      0  [original]여야 정치권이 사설 해병대 캠프 참사와 관련 한목소리를 냈다. 즉...
1      0  [original]고위 공무원과 대형병원간 검은 커넥션이 드러나 충격을 더하고 있다...
2      0  [original]신의 퀴즈:리부트가 충격적 진실로 시작해 소름 돋는 엔딩으로 최종...
3      0  [original]지역사회 개방, 투명성공공성 등 개선 노력 의지피력\r 교육부로부...
4      0  [original]부안수협 김진태 조합장이 오는 2월 22일 실시되는 제25대 수협...
데이터가 저장되었습니다.


In [5]:
# validation_machine_opinion.csv : VL_기계요약문_의견제시형_corrected_type1 ~ VL_기계요약문_의견제시형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    machine_type_data_path = base_dir_path + f'VL_기계요약문_의견제시형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(machine_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(machine_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['machine_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_machine_opinion = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_machine_opinion.shape)
print(data_df_validation_machine_opinion.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_machine_opinion.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_validation_machine_opinion.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (3467, 2)
   label                                               text
0      0  [original]국민연금이 2015년부터 헤지펀드에 투자하겠다는 계획을 밝혔다. ...
1      0  [original]성시윤 교육팀장 교육부가 유치원어린이집 방과후 영어 수업 금지 방...
2      0  [original]성윤모 산업통상자원부 장관이 28~29일 중국의 하와이로 불리는 ...
3      0  [original]우리나라 억대 연봉자가 1년 새 42%나 늘어 작년 말 기준 27...
4      0  [original]많은 취업 준비생이 국제기구 진출을 열망한다. 세계를 무대로 국제...
데이터가 저장되었습니다.


In [6]:
# validation_machine_free.csv : VL_기계요약문_자유형_corrected_type1 ~ VL_기계요약문_자유형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    machine_type_data_path = base_dir_path + f'VL_기계요약문_자유형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(machine_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(machine_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['machine_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_machine_free = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_machine_free.shape)
print(data_df_validation_machine_free.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_machine_free.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_validation_machine_free.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (2010, 2)
   label                                               text
0      0  [original]1 구 청소년보호법(2001. 5. 24. 법률 제6479호로 개...
1      0  [original]1 구 법인세법(1996. 12. 30. 법률 제5192호로 개정...
2      0  [original]구 의료법(2000. 1. 12. 법률 제6157호로 개정되기 전...
3      0  [original]1자기신체사고 및 무보험자동차상해보험은 모두 상해보험의 일종으로서...
4      0  [original]［1］ 영상물등급위원회가 등급분류 과정에서 음란성 여부에 관한 판...
데이터가 저장되었습니다.


In [7]:
# validation_human_fact.csv : VL_사람요약문_사실전달형_corrected_type1 ~ VL_사람요약문_사실전달형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'VL_사람요약문_사실전달형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_human_fact = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_human_fact.shape)
print(data_df_validation_human_fact.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_human_fact.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_validation_human_fact.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (454, 2)
   label                                               text
0      0  [original]30일 코스닥 지수는 보합권에서 상승 출발했다. 미국 연방공개시장...
1      0  [original]경기도 노후 하수관이 1만㎞에 달하며 지하안전 주의보가 발령된(본...
2      0  [original]정부와 여당인 더불어민주당이 증권거래세 폐지를 본격 추진하면서 주...
3      0  [original]엄재천 올해 장학생 280명 선발2억3천만원 지급 계획,  22일...
4      0  [original]전북경찰이 해마다 업무 중 불의의 사고를 당하는 것으로 나타났다....
데이터가 저장되었습니다.


In [8]:
# validation_human_opinion.csv : VL_사람요약문_의견제시형_corrected_type1 ~ VL_사람요약문_의견제시형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'VL_사람요약문_의견제시형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_human_opinion = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_human_opinion.shape)
print(data_df_validation_human_opinion.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_human_opinion.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_validation_human_opinion.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (426, 2)
   label                                               text
0      0  [original]01. 지방선거 지지 정당 질문 | 6월 13일 시도지사 등을 새...
1      0  [original]최악의 한 해였다. 실적은 급전 직하했고, 주가는 폭락했다. 이제...
2      0  [original]직장생활 초기에는 나이를 내세우며 아버지뻘이라고 자처하는 취재원을...
3      0  [original]일단 정부는 청년 창업 지원책으로 청년층 취업 문제와 창조경제 실...
4      0  [original]백화점과 마트자동차가전 등 각 업계가 총출동한 대한민국 동행세일이...
데이터가 저장되었습니다.


In [9]:
# validation_human_free.csv : VL_사람요약문_자유형_corrected_type1 ~ VL_사람요약문_자유형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    if type_num == 5 :
        continue
        
    human_type_data_path = base_dir_path + f'VL_사람요약문_자유형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            original_text = data['original_text']

            try:
                corrected_type = type_num

                if index < len(json_files) // percent:
                    summary_text = data['annotation']['corrected_summary'][f'corrected_type{type_num}']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': 0
                    })
                else:
                    summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
                    data_list.append({
                        'original': original_text,
                        'summary': summary_text,
                        'corrected_type': corrected_type
                    })
            except KeyError:
                summary_text = None  # corrected_type1가 없는 경우 None 처리

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['text'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

        #corrected_type 열을 'label'열로 변경
        type_data_df = type_data_df.rename(columns={'corrected_type': 'label'})

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_human_free = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_human_free.shape)
print(data_df_validation_human_free.head())

# csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_human_free.csv'

# 데이터프레임을 CSV 파일로 저장
data_df_validation_human_free.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (234, 2)
   label                                               text
0      0  [original]1 산업재해보상보험법상의 업무상 재해라 함은 업무수행중 그 업무에...
1      0  [original]도시재개발법(1995. 12. 29. 법률 제5116호로 전문 개...
2      0  [original]1 도시 및 주거환경정비법 제84조는  형법 제129조 내지 제1...
3      0  [original]1 생명보험계약은 사람의 생명에 관한 우연한 사고에 대하여 금전을...
4      1  [original]1보험계약자의 채무불이행시 계약보증금채권을 담보하는 계약이행보증보...
데이터가 저장되었습니다.


## 데이터 병합

In [38]:
# training 기계요약문 병합

training_base_dir_path = '../data/training/'

csv_paths = [
    training_base_dir_path + 'training_machine_fact.csv',
    training_base_dir_path + 'training_machine_opinion.csv',
    training_base_dir_path + 'training_machine_free.csv'
]

dataframes = [pd.read_csv(path) for path in csv_paths]
merged_training_machine_dataframe = pd.concat(dataframes, ignore_index=True)

# 결과 DataFrame을 새로운 CSV 파일로 저장
output_path = training_base_dir_path + 'training_machine_data.csv'
merged_training_machine_dataframe.to_csv(output_path, index=False)

print('병합이 완료되었습니다.')
print('shape:', merged_training_machine_dataframe.shape)
print(merged_training_machine_dataframe.head())


병합이 완료되었습니다.
shape: (72217, 2)
   label                                               text
0      0  [original]음성군이 여름방학을 맞은 어린이들에게 다양한 볼거리와 체험 행사를...
1      0  [original]창단 4년 만에 괄목할만한 대회 성적을 올리며 화제가 되고 있는 ...
2      0  [original]주방가전 전문 브랜드 하우스쿡이 가정의 달 이벤트를 진행한다. 5...
3      0  [original]대한축구협회(KFA)가 FIFA U-20 월드컵 준우승을 차지한 ...
4      0  [original]올해 2분기 전세계 스마트폰 판매량이 전년 대비 소폭 감소했다. ...


In [39]:
# training 사람요약문 병합

csv_paths = [
    training_base_dir_path + 'training_human_fact.csv',
    training_base_dir_path + 'training_human_opinion.csv',
    training_base_dir_path + 'training_human_free.csv'
]

dataframes = [pd.read_csv(path) for path in csv_paths]
merged_training_human_dataframe = pd.concat(dataframes, ignore_index=True)

# 결과 DataFrame을 새로운 CSV 파일로 저장
output_path = training_base_dir_path + 'training_human_data.csv'
merged_training_human_dataframe.to_csv(output_path, index=False)

print('병합이 완료되었습니다.')
print('shape:', merged_training_human_dataframe.shape)
print(merged_training_human_dataframe.head())


병합이 완료되었습니다.
shape: (8910, 2)
   label                                               text
0      0  [original]제이테크놀로지(옛 마제스타)가 카지노 자회사 매각을 완료해 재무건...
1      0  [original]글로벌 뷰티 브랜드 랑콤이 패션 매거진 <마리끌레르>와 함께한 배...
2      0  [original]법동초 시작관내 초 순회 진단. 대덕구가 지난 30일 우송대학교 ...
3      0  [original]경기파주시 최종환시장은 지난 4일 신설 마을버스 운행 첫날 시민들...
4      0  [original]배터리 셀을 든 최태원(가운데) 회장이 김진영 배터리생산기술본부장...


In [40]:
# training 요약문 전체 병합

dataframes = [merged_training_machine_dataframe, merged_training_human_dataframe]
merged_training_dataframe = pd.concat(dataframes, ignore_index=True)

# 결과 DataFrame을 새로운 CSV 파일로 저장
output_path = training_base_dir_path + 'training_data.csv'
merged_training_dataframe.to_csv(output_path, index=False)

print('병합이 완료되었습니다.')
print('shape:', merged_training_dataframe.shape)
print(merged_training_dataframe.head())


병합이 완료되었습니다.
shape: (81127, 2)
   label                                               text
0      0  [original]음성군이 여름방학을 맞은 어린이들에게 다양한 볼거리와 체험 행사를...
1      0  [original]창단 4년 만에 괄목할만한 대회 성적을 올리며 화제가 되고 있는 ...
2      0  [original]주방가전 전문 브랜드 하우스쿡이 가정의 달 이벤트를 진행한다. 5...
3      0  [original]대한축구협회(KFA)가 FIFA U-20 월드컵 준우승을 차지한 ...
4      0  [original]올해 2분기 전세계 스마트폰 판매량이 전년 대비 소폭 감소했다. ...


In [13]:
# validation 기계요약문 병합

validation_base_dir_path = '../data/validation/'

csv_paths = [
    validation_base_dir_path + 'validation_machine_fact.csv',
    validation_base_dir_path + 'validation_machine_opinion.csv',
    validation_base_dir_path + 'validation_machine_free.csv'
]

dataframes = [pd.read_csv(path, encoding='utf-8') for path in csv_paths]
merged_validation_machine_dataframe = pd.concat(dataframes, ignore_index=True)

# 결과 DataFrame을 새로운 CSV 파일로 저장
output_path = validation_base_dir_path + 'training_machine_data.csv'
merged_validation_machine_dataframe.to_csv(output_path, index=False)

print('병합이 완료되었습니다.')
print('shape:', merged_validation_machine_dataframe.shape)
print(merged_validation_machine_dataframe.head())


병합이 완료되었습니다.
shape: (9014, 2)
   label                                               text
0      0  [original]여야 정치권이 사설 해병대 캠프 참사와 관련 한목소리를 냈다. 즉...
1      0  [original]고위 공무원과 대형병원간 검은 커넥션이 드러나 충격을 더하고 있다...
2      0  [original]신의 퀴즈:리부트가 충격적 진실로 시작해 소름 돋는 엔딩으로 최종...
3      0  [original]지역사회 개방, 투명성공공성 등 개선 노력 의지피력\r 교육부로부...
4      0  [original]부안수협 김진태 조합장이 오는 2월 22일 실시되는 제25대 수협...


In [11]:
# validation 사람요약문 병합

validation_base_dir_path = '../data/validation/'

csv_paths = [
    validation_base_dir_path + 'validation_human_fact.csv',
    validation_base_dir_path + 'validation_human_opinion.csv',
    validation_base_dir_path + 'validation_human_free.csv'
]

dataframes = [pd.read_csv(path, encoding='utf-8') for path in csv_paths]
merged_validation_human_dataframe = pd.concat(dataframes, ignore_index=True)

# 결과 DataFrame을 새로운 CSV 파일로 저장
output_path = validation_base_dir_path + 'training_human_data.csv'
merged_validation_human_dataframe.to_csv(output_path, index=False)

print('병합이 완료되었습니다.')
print('shape:', merged_validation_human_dataframe.shape)
print(merged_validation_human_dataframe.head())


병합이 완료되었습니다.
shape: (1114, 2)
   label                                               text
0      0  [original]30일 코스닥 지수는 보합권에서 상승 출발했다. 미국 연방공개시장...
1      0  [original]경기도 노후 하수관이 1만㎞에 달하며 지하안전 주의보가 발령된(본...
2      0  [original]정부와 여당인 더불어민주당이 증권거래세 폐지를 본격 추진하면서 주...
3      0  [original]엄재천 올해 장학생 280명 선발2억3천만원 지급 계획,  22일...
4      0  [original]전북경찰이 해마다 업무 중 불의의 사고를 당하는 것으로 나타났다....
